In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

In [4]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Reduce the size of the dataset
num_samples = 1000
train_images, train_labels = train_images[:num_samples], train_labels[:num_samples]


In [5]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 227x227 (AlexNet input size)
    image = tf.image.resize(image, (227, 227))
    return image, label

# Prepare the datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_ds = (train_ds
            .map(process_images)
            .shuffle(buffer_size=num_samples)
            .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
           .map(process_images)
           .batch(batch_size=32, drop_remainder=True))


In [6]:
# Define the AlexNet model
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 27, 27, 96)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_8 (Conv2D)           (None, 13, 13, 384)      

In [7]:
# Define the log directory for TensorBoard
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)


In [2]:
# Train the model
model.fit(train_ds,
          epochs=5,
          validation_data=test_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)


Epoch 1/5
31/31 [==============================] - 138s 5s/step - loss: 2.3085 - accuracy: 0.1008 - val_loss: 2.3030 - val_accuracy: 0.1053
Epoch 2/5
31/31 [==============================] - 165s 5s/step - loss: 2.3093 - accuracy: 0.0958 - val_loss: 2.3005 - val_accuracy: 0.1106
Epoch 3/5
31/31 [==============================] - 159s 5s/step - loss: 2.3019 - accuracy: 0.1119 - val_loss: 2.2985 - val_accuracy: 0.1211
Epoch 4/5
31/31 [==============================] - 184s 6s/step - loss: 2.3002 - accuracy: 0.1310 - val_loss: 2.2966 - val_accuracy: 0.1279
Epoch 5/5
312/312 [==============================] - 152s 489ms/step - loss: 2.2944 - accuracy: 0.1415
Test Accuracy: 0.14152644574642181
